In [6]:
import pandas as pd, os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
os.environ["OPENAI_API_KEY"] = "sk-proj-If7k9gDIzcM7HB9HcARRT3BlbkFJc1v6B38x9HNMxpbNIqHQ"

In [10]:
data = pd.read_csv("leads.csv")

In [36]:
def read_excel_dynamic(filepath):
	df = pd.read_excel(filepath)
 
	non_empty_row = next((i for i, row in df.iterrows() if not row.isnull().all()), None)
	if non_empty_row is not None:
		df.columns = df.iloc[non_empty_row]
		df = df.iloc[non_empty_row+1:]
		non_empty_col = df.notna().any(axis=0)
		df = df.loc[:, non_empty_col]
	else:
		print("Excel file seems to be empty!")
		df = pd.DataFrame()
	return df

filepath = 'leads.xlsx'
df = read_excel_dynamic(filepath)
df.head(n=15)


3,Name,Job Title,Organizaton,Company Size,Department,Project Title,Looking For,Lead Response
4,Yaqub Qamar Ad Din Dibiazah,Director Engineering,Imaarat Tech,50-100,Engineering,CentDoc,"In the past, the company encountered an issue ...",Thanks for reaching out. Our procurement team ...
5,Joe Steve,CTO,Anz LLC,25-50,Content,ConSys,Anz LLC is a company that creates SEO-enabled ...,Thank you for contacting us. Please find the a...
6,Sophia Rodriguez,Lead Developer,TechSprint Innovations,250-400,Engineering,AutoCode,TechSprint Innovations is focused on automatin...,"For the scope of this project, we require a so..."
7,Ayesha Raza,Marketing Manager,LuxeDrive Rentals,7,-,CarElite,LuxeDrive Rentals specializes in providing lux...,"Hi. As we're starting out, our budget for this..."


In [ ]:
['Goal: Craft an email to send to a potential lead. This email should be concise, informative, and engaging. The tone of the email should be human like. It should inquire more details about the scope and budget of project they have in mind. The email should have a brief subject that\'s less than 30 characters. The email body shouldnt 
be longer than 3 paragraphs. The email should not use any flowery or flattery language.\n\nUse the below provided information to craft the email.\nInformation about the lead:\n- Name: Yaqub Qamar Ad Din Dibiazah\n- Job Title: Director Engineering\n- Organization: Imaarat Tech\n- Company Size: 50-100\n- Department: Engineering\n- Project Title: CentDoc\n- Looking for: In the past, the company encountered an issue where their product manuals became out of sync with the software documentation. As the development teams continued to add new features and refine existing ones, they also updated the documentation. However, this required manual synchronization between the 
product manuals and the software documentation.\n\nTo address this problem, the company has envisioned a solution called "CentDoc". This tool will automatically detect any inconsistencies between the product manuals and software documentation. It will then remove outdated information from the manuals using the software documentation, 
but only after receiving explicit approval from the user.', "Goal: Craft an email to send to a potential lead. This email should be concise, informative, and engaging. The tone of the email should be human like. It should inquire more details about the scope and budget of project they have in mind. The email should have a brief subject that's less than 30 characters. The email body shouldnt be longer than 3 paragraphs. The email should not use 
any flowery or flattery language.\n\nUse the below provided information to craft the email.\nInformation about the lead:\n- Name: Joe Steve\n- Job Title: CTO\n- Organization: Anz LLC\n- Company Size: 25-50\n- Department: Content\n- Project Title: ConSys\n- Looking for: Anz LLC is a company that creates SEO-enabled internet content for small businesses. They have a team of 20 members who write content for various products offered by different businesses. The team now plans to utilize generative AI to expedite the writing process, which will facilitate scaling for them.", "Goal: Craft an email to send to a potential lead. This email should be concise, informative, and engaging. The tone of the email should be human like. It should inquire more details about the scope and budget of project they have in mind. The email should have a brief subject that's less than 30 characters. The email body shouldnt be longer than 3 paragraphs. The email should not use any flowery or flattery language.\n\nUse the below provided information to craft the email.\nInformation about the lead:\n- Name: Sophia Rodriguez\n- Job Title: Lead Developer\n- Organization: TechSprint Innovations\n- Company Size: 250-400\n- Department: Engineering\n- Project Title: AutoCode\n- Looking for: TechSprint Innovations is focused on automating the coding process to enhance development speed and accuracy. As the Lead Developer, Sophia Rodriguez spearheads the implementation 
of AutoCode, an innovative tool designed to automatically generate code snippets based on predefined patterns. This aims to streamline the coding workflow and boost overall development efficiency.", "Goal: Craft an email to 
send to a potential lead. This email should be concise, informative, and engaging. The tone of the email should 
be human like. It should inquire more details about the scope and budget of project they have in mind. The email should have a brief subject that's less than 30 characters. The email body shouldnt be longer than 3 paragraphs. The email should not use any flowery or flattery language.\n\nUse the below provided information to craft the 
email.\nInformation about the lead:\n- Name: Ayesha Raza\n- Job Title: Marketing Manager\n- Organization: LuxeDrive Rentals\n- Company Size: 7\n- Department: - \n- Project Title: CarElite\n- Looking for: LuxeDrive Rentals specializes in providing luxury car rental services, offering an exclusive fleet of high-end vehicles. Isabella Thompson, the Marketing Manager, is spearheading the initiative to establish a strong online presence. The company envisions a complete mobile solution, CarElite, tailored to showcase their premium car collection, facilitate seamless bookings, and enhance the overall customer experience."]

In [11]:
prompt = """Goal: Craft an email to send to a potential lead. This email should be concise, informative, and engaging. The tone of the email should be human like. It should inquire more details about the scope and budget of project they have in mind. The email should have a brief subject that\'s less than 30 characters. Use HR and hr@antematter.so in Regards section of the email. The email body shouldnt 
be longer than 3 paragraphs. The email should not use any flowery or flattery language.\n\nUse the below provided information to craft the email.\nInformation about the lead:\n- Name: Yaqub Qamar Ad Din Dibiazah\n- Job Title: Director Engineering\n- Organization: Imaarat Tech\n- Company Size: 50-100\n- Department: Engineering\n- Project Title: CentDoc\n- Looking for: In the past, the company encountered an issue where their product manuals became out of sync with the software documentation. As the development teams continued to add new features and refine existing ones, they also updated the documentation. However, this required manual synchronization between the 
product manuals and the software documentation.\n\nTo address this problem, the company has envisioned a solution called "CentDoc". This tool will automatically detect any inconsistencies between the product manuals and software documentation. It will then remove outdated information from the manuals using the software documentation, 
but only after receiving explicit approval from the user."""

In [38]:
llm = ChatOpenAI(temperature=0.2, model_name='gpt-3.5-turbo', max_tokens=300)

In [9]:
messages = [
    ("system", prompt)
]
ret = llm.invoke(prompt)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************6USa. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

## Escalator Agent

In [8]:
prompt = """
Context: The lead was sent an opening email on their given information which is in `Looking For` section.
Based on which the lead has responded to the email. The lead response to the email is in Lead Response section.

Goal: If the lead response to the email has asked for more information or has mentioned the scope and budget,
then reply only "Escalated". Else if in the lead response to the email, scope or budget is not 
mentioned, then send an email to th00e mentioned mail in the lead response, if not mentioned then to the lead 
themselves. The email should be concise, informative, and engaging. The tone of the email should be human like. The email should inquire more details about the scope and budget of project they have in mind. The email should have a brief subject that's less than 30 characters. Use HR and hr@antermatter.so in regards section of the email. The email body shouldnt be longer than 3 paragraphs. The email should not use any flowery or flattery language. Use the below provided information to craft the email or response.

Information about the lead:
- Name: Yaqub Qamar Ad Din Dibiazah
- Job Title: Director Engineering
- Organization: Imaarat Tech
- Company Size: 50-100
- Department: Engineering
- Project Title: CentDoc
- Looking for: In the past, the company encountered an issue where their product manuals became out of sync with the software documentation. As the development teams continued to add new features and refine existing ones, they also updated the documentation. However, this required manual synchronization between the product manuals and the software documentation.

To address this problem, the company has envisioned a solution called ""CentDoc"". This tool will automatically detect any inconsistencies between the product manuals and software documentation. It will then remove outdated information from the manuals using the software documentation, but only after receiving explicit approval from the user.
- Lead Response: Thanks for reaching out. Our procurement team is actively looking for potential consultants. Can you please get in touch with them at procure@centdoc.io
"""

In [44]:
prompt = """
Goal: Refine the understanding of the lead's project needs and determine the next steps.

Based on Lead Response:
1. budget & scope Provided:
Action: Escalate the lead to the admin for further qualification and potential sales engagement.
Response: "Escalated"

2. Missing Information (budget or scope):
Action: Prompt the user to provide the missing details. Note: Use HR and hr@antematter.so in the Regards section of the email.
Budget Missing: "Thanks for sharing the project scope, [Name]! To get a better idea of how we can best assist you, could you share your estimated budget for this project?"
Scope Missing: "Thanks for letting me know your budget, [Name]! To understand your needs more clearly, could you briefly describe the project scope or what you're hoping to achieve?"

3. Requesting More Information:
Action: Escalate the lead to the admin for further nurturing and potential information sharing.
Response: "Escalated"

Lead Information and Response To Opener Email:
- Name: Ayesha Raza
- Job Title: Marketing Manager
- Organization: LuxeDrive Rentals
- Company Size: 7
- Department: -
- Project Title: CarElite
- Looking for: LuxeDrive Rentals specializes in providing luxury car rental services, offering an exclusive fleet of high-end vehicles. Isabella Thompson, the Marketing Manager, is spearheading the initiative to establish a strong online presence. The company envisions a complete mobile solution, CarElite, tailored to showcase their premium car collection, facilitate seamless bookings, and enhance the overall customer experience.
- Lead Response: Hi. As we're starting out, our budget for this is $25k and we want a mobile app.
"""

In [45]:
messages = [
    ("system", prompt)
]
ret = llm.invoke(prompt)

In [46]:
print(ret.content)

Based on the lead's response, it seems that the budget and scope have been provided. Therefore, the lead can be escalated to the admin for further qualification and potential sales engagement. 

Response: "Escalated"
